# 【NLP 自然語言處理】03. FaceBook社團貼文爬蟲 Bonus

In [32]:
# !pip install selenium pandas fake_useragent python-dotenv tqdm

## 瀏覽器設定：允許所有網站通知行為

In [5]:
from selenium import webdriver
from fake_useragent import UserAgent

In [6]:
chrome_options = webdriver.ChromeOptions()

# 允許所有網站通知行為
chrome_options.add_experimental_option(
    "prefs", 
    {
        "profile.default_content_setting_values.notifications": 1
    }
)


## 01. 使用者代理 User-Agent

在爬蟲時，最常遇到的其中一個問題就是網頁阻擋。對方為了保護自己的網站，避免資源消耗，可能透過鎖 ip 的方式阻擋爬蟲程式。
而 User-Agent 記錄了目前瀏覽網站的瀏覽器、作業系統是什麼。在對網站發送請求時，標頭都會攜帶 User-Agent。

當然，對於網站主來說，對方也可以針對 User-Agent 的內容物來判斷是否為爬蟲程式，並建立反爬蟲機制。尤其 requests 的請求方式更是容易被偵測出來。

雖然 selenium 會提供預設的 User-Agent，但短時間內的大量請求，還是很可能被阻擋下來。

In [8]:
ua = UserAgent()

random_user_agent = ua.random

在 Python 中，我們可以利用 fake_useragent 套件，來隨機生成 User-Agent。幫助我們的爬蟲程式偽裝成一般使用者。

In [9]:
# 使用 fake_useragent 生成的 User-Agent 设置到 ChromeOptions 中
chrome_options.add_argument(f"user-agent={random_user_agent}")


## 02. 環境變數

在程式開發階段時，通常會將機敏資訊存入環境變數，例如：帳號、密碼、金鑰 … 等。避免將機敏資訊連同程式碼一起部屬，確保資訊的使用是安全的！

In [16]:
from dotenv import load_dotenv, find_dotenv
import os

load_dotenv(find_dotenv())

facebook_mail = os.environ.get("FACEBOOK_MAIL")
facebook_password = os.environ.get("FACEBOOK_PASSWORD")

## 啟動 webdriver 並登入 FB

In [22]:
from selenium.webdriver.common.by import By
import time

driver = webdriver.Chrome(options = chrome_options)

# 設定視窗大小
driver.set_window_size(800, 800)

# Facebook 登入
login_url = 'https://www.facebook.com/login/device-based/regular/login/?login_attempt=1&next=https%3A%2F%2Fwww.facebook.com'
driver.get(login_url)

email_element = driver.find_element(By.CSS_SELECTOR,'#email_container input')
password_element = driver.find_element(By.CSS_SELECTOR,'._55r1._1kbt input')

email_element.send_keys(facebook_mail)
password_element.send_keys(facebook_password)

login_button = driver.find_element(By.CSS_SELECTOR, '#loginbutton')
login_button.click()

time.sleep(3)

# 前往指定社團頁面
url = 'https://www.facebook.com/groups/traveler168'

driver.get(url)

In [53]:
from bs4 import BeautifulSoup
import pandas as pd
import re

js_script =  '''
    let elements = document.querySelectorAll(".x9f619.x1n2onr6.x1ja2u2z.x1s85apg[hidden]");
    for (let element of elements) {
        let parentElement = element.closest(".x1yztbdb.x1n2onr6.xh8yej3.x1ja2u2z");
        parentElement.parentNode.removeChild(parentElement);
    }
'''

def getElement(driver):
    page_content = driver.page_source
    soup = BeautifulSoup(page_content, 'html.parser')
    elements = soup.select('.x1yztbdb.x1n2onr6.xh8yej3.x1ja2u2z')
    return elements

def getPostData(elements):
    name_list, like_list, date_list, link_list, article_list = [], [], [], [], []
    for element in elements:
        try:
            name = element.select('.xt0psk2 span')[0].text
            name_list.append(name)
        except:
            continue

        try:
            like = element.select('.xt0b8zv.x2bj2ny.xrbpyxo.xl423tq span.x1e558r4')[0].text
            like_list.append(like)
        except:
            like_list.append(0)

        try:
            date_time = element.select('.x1i10hfl.xjbqb8w.x1ejq31n.xd10rxx.x1sy0etr.x17r0tee.x972fbf.xcfux6l.x1qhh985.xm0m39n.x9f619.x1ypdohk.xt0psk2.xe8uvvx.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.xexx8yu.x4uap5.x18d9i69.xkhd6sd.x16tdsg8.x1hl2dhg.xggy1nq.x1a2a7pz.x1heor9g.xt0b8zv.xo1l8bm span')[0].text
            date_list.append(date_time)
        except:
            continue

        try:
            link = element.select('.x1i10hfl.xjbqb8w.x1ejq31n.xd10rxx.x1sy0etr.x17r0tee.x972fbf.xcfux6l.x1qhh985.xm0m39n.x9f619.x1ypdohk.xt0psk2.xe8uvvx.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.xexx8yu.x4uap5.x18d9i69.xkhd6sd.x16tdsg8.x1hl2dhg.xggy1nq.x1a2a7pz.x1heor9g.xt0b8zv.xo1l8bm')[0].get('href')
            base_url = re.match(r"(https://www\.facebook\.com/groups/traveler168/posts/\d+)", link)
            if base_url:
                link_list.append(base_url.group(1))
            else:
                link_list.append(link)
        except:
            continue

        try:
            article = element.select_one('.x193iq5w.xeuugli.x13faqbe.x1vvkbs.x1xmvt09.x1lliihq.x1s928wv.xhkezso.x1gmr53x.x1cpjm7i.x1fgarty.x1943h6x.xudqn12.x3x7a5m.x6prxxf.xvq8zen.xo1l8bm.xzsf02u.x1yc453h')
            article_list.append(article.get_text())
        except:
            article_list.append('')

    df = pd.DataFrame({
        '作者': name_list,
        '讚數': like_list,
        '時間': date_list,
        '連結': link_list,
        '貼文內容': article_list
    })
    return df

## 03. 任務進度條

當爬取大量網頁資料時，可能是耗時的，進度條可以幫助我們查看目前的執行進度。常見的套件有 `alive-progress`、`tqdm` 。本文以 `tqdm` 舉例：

- Step 1. 初始化 tqdm 的進度條

- Step 2. 更新進度條至新的完成數量
    - `amount` 是這一輪後的資料量；
    - `pbar.n` 是前一輪紀錄的資料量；
    - `pbar.update(amount - pbar.n)` 更新這一輪的資料筆數道進度條。

In [ ]:
from tqdm import tqdm
import time

data_capacity = 1000

# 初始化 tqdm 的進度條
pbar = tqdm(total = data_capacity)

result_df = pd.DataFrame()

amount = 0
while amount < data_capacity:
    
    counter = 0
    while counter <= 3:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(0.3)
        counter += 1

    try:
        target_class = '.x1i10hfl.xjbqb8w.x1ejq31n.xd10rxx.x1sy0etr.x17r0tee.x972fbf.xcfux6l.x1qhh985.xm0m39n.x9f619.x1ypdohk.xt0psk2.xe8uvvx.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.xexx8yu.x4uap5.x18d9i69.xkhd6sd.x16tdsg8.x1hl2dhg.xggy1nq.x1a2a7pz.xt0b8zv.xzsf02u.x1s688f'
        targets = driver.find_elements(By.CSS_SELECTOR, target_class)
        for target in targets:
            try:
                if "查看更多" in target.get_attribute('textContent'):
                    target.click()
                    time.sleep(0.3)
            except:
                continue
    except Exception as e:
        print(e)
        pass
    
    elements = getElement(driver)
    post_df = getPostData(elements)

    result_df = pd.concat([result_df, post_df])

    time.sleep(0.3)

    driver.execute_script(js_script)

    result_df['貼文內容'] = result_df['貼文內容'].astype(str)
    amount = len(result_df[~result_df['貼文內容'].str.contains('查看更多')]['連結'].unique())

    # 更新進度條至新的完成數量
    pbar.update(amount - pbar.n)

pbar.close()  
driver.close()

100%|██████████| 1000/1000 [17:34<00:00,  1.05s/it]


儲存

In [ ]:
dataset = result_df[~result_df['貼文內容'].str.contains('查看更多')]

dataset = dataset.drop_duplicates(subset = '連結').reset_index(drop = True)

dataset

,作者,讚數,時間,連結,貼文內容
0,鄭玉潔,0,3小時,https://www.facebook.com/groups/traveler168/po...,最夢幻溫泉山莊藏身萬坪原始林中，僅18間客房都為獨棟小屋都有專屬戶外露天溫泉風呂泡湯賞雪同時...
1,跟著領隊玩,2,4小時,https://www.facebook.com/groups/traveler168/po...,Google高達4.3顆星！顛覆你客家小吃的印象，北埔的客家桌菜，道道是經典，吃過的都說讚，...
2,Amy Lin,0,4小時,https://www.facebook.com/groups/traveler168/po...,#板橋美食海鮮夠鮮美才敢清蒸！蒸天下蒸氣火鍋海鮮餐廳非常適合聚餐宴客，還有波士頓龍蝦可以品嘗...
3,宗韓,0,4小時,https://www.facebook.com/groups/traveler168/po...,"藝綺地中海私廚餐酒館, 復古貴族風, 香煎特級牛排份量厚度都很威, 約會很推薦, 一起享受浪..."
4,謝露西,0,4小時,https://www.facebook.com/groups/traveler168/po...,有人還沒跟上這波免簽的嗎??大概倒數一個月囉~~~(希望可以再延長啊啊啊啊!!!
...,...,...,...,...,...
995,Lucky hamburger ！,0,3月22日,https://www.facebook.com/groups/traveler168/po...,在地土城人～還沒吃過 Lucky 漢堡嗎？ 迫不及待趕快來吃看看吧～ 本月銷售月冠軍 熔...
996,Sinian Li,2,3月22日,https://www.facebook.com/groups/traveler168/po...,補財庫_招財小物一次看：『紫南宮』求發財金美食之旅_有拜有保庇_財富攏總來紫南宮：南投縣竹山...
997,任秋磊,0,3月22日,https://www.facebook.com/groups/traveler168/po...,超愛義大利羅馬假期義大利10日游全程4星級飯店無車購/無購物特別加碼贈送: 轉接插頭 / 行...
998,高樂天,4,3月22日,https://www.facebook.com/groups/traveler168/po...,原本隱身在北安橋下大腸包小腸，被檢舉才搬遷現址開啟實體店面「原北安橋下黑輪攤」。「原北安橋下...


## 04. 時間格式調整

從 Facebook 社團爬取的貼文時間，格式並不統一。例如：「1小時」、「1月24日」、「1天」。

In [48]:
from datetime import datetime, timedelta

def convert_time_to_date(time_str):
    today = datetime.now()
    if '小時' in time_str or '分鐘' in time_str:
        hours_ago = int(re.findall(r'\d+', time_str)[0])
        target_date = today - timedelta(hours = hours_ago)
    elif '天' in time_str:
        days_ago = int(re.findall(r'\d+', time_str)[0])
        target_date = today - timedelta(days = days_ago)
    elif '月' in time_str:
        # Assuming the format is '4月17日下午7:55', only extract month and day
        month, day = map(int, re.findall(r'\d+', time_str[:time_str.index('日') + 1]))
        target_date = datetime(year = 2024, month = month, day = day)
    else:
        return time_str

    return target_date.strftime('2024-%m-%d')

In [51]:
dataset['時間'] = dataset['時間'].apply(convert_time_to_date)

In [52]:
dataset.to_csv('./data/fb_post_02.csv', index = False)